## Setup Notebook, Install


In [ ]:
!git clone https://github.com/JonathanFly/bark.git
%cd bark
!pip install -r requirements-pip.txt
!pip install encodec rich-argparse
!pip install librosa pydub

## Run Once Per Notebook Restart (if files still exist)

In [ ]:
import os
import time
from bark_infinity import config
import numpy as np

logger = config.logger
logger.setLevel("WARNING")

from bark_infinity import generation
from bark_infinity import api

import rich
from rich import print
from rich import pretty
from rich.pretty import pprint
from rich import inspect

from pydub import AudioSegment
import ipywidgets as widgets
from IPython.display import display, Audio
from io import BytesIO

# None of this code, just fiddlign with Colab stuff
# Just to save Colab with outputs and float32 wavs are GIGANTO
# actually this doesn't work, the iPython widget converts it back to float32? or I messed up

def display_audio_int16_but(audio_arr_segments, file_name, sample_rate=generation.SAMPLE_RATE,  width='200px'):
    file_name_label = widgets.Label(value=f"Playing: {file_name}")
    file_name_label.layout.width = width
    audio_data_int16 = audio_arr_segments
    if isinstance(audio_data_int16, list):
        audio_data_int16 = np.concatenate(audio_data_int16)        

    #audio_data_int16 = np.int16(audio_data_int16 * np.iinfo(np.int16).max)


    audio_widget = Audio(audio_data_int16, rate=sample_rate)
    display(file_name_label, audio_widget)
    

def on_button_click(button):
    audio_data, sample_rate = librosa.load(button.wav_path, sr=None)
    file_name = os.path.basename(button.wav_path)
    display_audio_int16_but(audio_data,file_name, sample_rate)


def display_wav_files(directory):
    subdirs, wav_files = [], []
    
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        
        if os.path.isfile(item_path) and item_path.endswith('.wav'):
            wav_files.append(item_path)
        elif os.path.isdir(item_path):
            subdirs.append(item_path)

    wav_files.sort(key=lambda x: os.path.basename(x))

    for wav_file in wav_files:

        filename = os.path.basename(wav_file)
        print(f" {filename}")
        button = widgets.Button(description=f"Play {filename}")
        button.wav_path = wav_file  
        button.on_click(on_button_click)
        display(button)

    for subdir in sorted(subdirs):
        print(f"<{subdir}>")
        display_wav_files(subdir)



## Generate


### Choose Bark Models

In [ ]:
generation.OFFLOAD_CPU = True # On your home system set to True probably, but Colab GPU should have plenty of memory for all three models
generation.preload_models() # Optional, will lazy load if not preloaded. First time run in New Colab has to download models

In [ ]:
text = """
Hark! I, the phantom visage of Edward Teach, rise from the 
abyss, forever bound to the briny depths. With me, brave the tumultuous seas, claim treasures untold,
and send foes to their watery doom.
"""

In [53]:
import os
import time
from bark_infinity import config
import numpy as np

logger = config.logger
logger.setLevel("WARNING")

from bark_infinity import generation
from bark_infinity import api

import rich
from rich import print
from rich import pretty
from rich.pretty import pprint
from rich import inspect


from pydub import AudioSegment
import ipywidgets as widgets
from IPython.display import display, Audio
from io import BytesIO
from tqdm import tqdm


# For split set split_character_goal_length and split_character_max_length
kwargs = {}

kwargs = config.load_all_defaults()
#kwargs['text_prompt'] = text
kwargs['hoarder_mode'] = True
kwargs["output_dir"] = 'bark_samples'
kwargs["history_prompt"] = None
# kwargs["single_starting_seed"] = None # 
# If you set seed you might want manually call generation.set_seed(-1) after to disable deterministic generation settings 
# I'm not cleaning up after this paramater at the moment and I'm not sure on other side effects
kwargs["stable_mode_interval"] = 1 # 0 for continous, 2,3,4 for mixed
kwargs["split_character_goal_length"] = 145
kwargs["split_character_max_length"] = 190
# kwargs["output_iterations"] = 1
kwargs["add_silence_between_segments"] = 0.0 # See: https://github.com/suno-ai/bark/blob/main/notebooks/long_form_generation.ipynb but not great for songs or stable_mode_interval 0
kwargs["semantic_min_eos_p"] = 0.2 # 0.20 is default, lower means more likely to stotp


# not sure on overall effect so far from these, but for example:
kwargs["semantic_top_k"] = None 
kwargs["semantic_top_p"] = None
kwargs["coarse_top_k"] = None
kwargs["coarse_top_p"] = None


In [ ]:


from rich import prompt, print,inspect
pprint(kwargs)

In [54]:
kwargs['history_prompt'] = "custom_speakers/hark_woman.npz"
text = """With me, brave the tumultuous seas, claim treasures untold, and send foes to their watery 
doom"""
kwargs['text_prompt'] = text
kwargs['single_starting_seed'] = None
generation.set_seed(37)
logger.setLevel("DEBUG")

In [55]:
api.gradio_try_to_cancel = False

In [12]:
kwargs['output_dir'] = "long"
generation.set_seed(37)
full_generation_segments, audio_arr_segments, final_filename_will_be = api.generate_audio_long(**kwargs)

set_seed Enabling deterministic algorithms                                                        ]8;id=368965;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=553125;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#929\929]8;;\

set_seed Set seed to 37                                                                           ]8;id=647827;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=697756;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#944\944]8;;\

generate_audio_long {'kwargs': {'text_prompt': 'With me, brave the tumultuous seas, claim treasures      ]8;id=540035;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=37163;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#507\507]8;;\
untold, and send foes to their watery \ndoom', 'list_speakers': None, 'dry_run': False,                            
'text_splits_only': False, 'history_prompt': 'custom_speakers/hark_woman.npz', 'prompt_file': None,                
'split_input_into_separate_prompts_by': None, 'split_input_into_separate_prompts_by_value': None,                  
'always_save_speaker': True, 'output_iterations': 1, 'output_filename': None, 'output_dir': 'long',                
'hoarder_mode': True, 'extra_stats': False, 'text_use_gpu': True, 'text_use_small': False,                         
'coarse_use_gpu': True, 'coarse_use_small': False, 'fine_use_gpu': True, 'fine_use_small': False,                  
'codec_use_gpu': True, 'force_reload': False, 'GLOBAL_ENABLE_MPS': None, 'USE_SMALL_MODELS': None,                 
'OFFLOAD_CPU': None, 'text_temp': 0.7, 'waveform_temp': 0.7, 'confused_travolta_mode': None, 'silent':             
False, 'seed': None, 'stable_mode_interval': 1, 'single_starting_seed': None,                                      
'split_character_goal_length': 145, 'split_character_max_length': 190, 'split_character_jitter': 0,                
'add_silence_between_segments': 0.0, 'split_each_text_prompt_by': None,                                            
'split_each_text_prompt_by_value': None, 'extra_confused_travolta_mode': None,                                     
'semantic_history_starting_weight': 1.0, 'semantic_history_future_weight': 1.0,                                    
'semantic_prev_segment_weight': 0.5, 'coarse_history_starting_weight': 1.0,                                        
'coarse_history_future_weight': 0.5, 'coarse_prev_segment_weight': 0.5, 'fine_history_starting_weight':            
1.0, 'fine_history_future_weight': 0.0, 'fine_prev_segment_weight': 0.0,                                           
'custom_audio_processing_function': None, 'use_smaller_models': False, 'semantic_temp': 0.7,                       
'semantic_top_k': None, 'semantic_top_p': None, 'semantic_min_eos_p': 2, 'semantic_max_gen_duration_s':            
None, 'semantic_allow_early_stop': True, 'semantic_use_kv_caching': True, 'semantic_seed': None,                   
'semantic_history_oversize_limit': None, 'coarse_temp': 0.7, 'coarse_top_k': None, 'coarse_top_p': None,           
'coarse_max_coarse_history': 630, 'coarse_sliding_window_len': 60, 'coarse_kv_caching': True,                      
'coarse_seed': None, 'coarse_history_time_alignment_hack': -2, 'fine_temp': 0.5, 'fine_seed': None,                
'render_npz_samples': False, 'loglevel': 'WARNING'}}                                                               

    () Segment Breakdown                                                                                           
┏━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Words ┃ Time Est ┃ Splitting long text aiming for 145 chars max 190                                         ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ 16    │ 6.40 s   │ With me, brave the tumultuous seas, claim treasures untold, and send foes to their       │
│   │       │ 94 chars │ watery doom                                                                              │
└───┴───────┴──────────┴──────────────────────────────────────────────────────────────────────────────────────────┘

write_seg_npz   .npz saved to long/With_me_brave_t-SPK-hark_woman_3.wav_initial_prompt.npz               ]8;id=827442;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=113095;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#316\316]8;;\

...

100%|██████████| 2/2 [00:04<00:00,  2.20s/it]


generate_audio_long stable_mode_interval: None of 1                                                      ]8;id=552246;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=649487;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#666\666]8;;\

write_seg_npz   .npz saved to long/With_me_brave_t-SPK-hark_woman_3.wav                                  ]8;id=9210;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=7946;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#316\316]8;;\

write_seg_wav   .wav saved to long/With_me_brave_t-SPK-hark_woman_3.wav                                  ]8;id=71943;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=741508;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#325\325]8;;\

Saved to long/With_me_brave_t-SPK-hark_woman_3.wav


In [14]:
Audio(audio_arr_segments, rate=generation.SAMPLE_RATE) 

In [11]:
inspect(full_generation_segments)

╭──────────────────────────────────────────── <class 'list'> ─────────────────────────────────────────────╮
│ Built-in mutable sequence.                                                                              │
│                                                                                                         │
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                   │ │
│ │ │   {                                                                                               │ │
│ │ │   │   'semantic_prompt': array([3589,   17,  298, 9182, 6948, 6948,   92,   28,  148,  234, 3825, │ │
│ │ │      5947,  171,   41,  171,  181,   59,   28,   28, 7313,   41, 1298,                            │ │
│ │ │      8407, 9054, 9054, 1896, 1896, 7433, 4869, 7858, 5566,  812,   23,                            │ │
│ │ │   │   171,  492,  492,   28,   28,  107, 8218, 1879, 1879, 2359, 8141,                            │ │
│ │ │      8141, 6997, 6997, 6997, 2199,  620,   59,   59,   28,   28,   28,                            │ │
│ │ │   │    28,   28,   28,  107,  385, 2954, 9381, 6319,  230,   10, 8849,                            │ │
│ │ │      7010, 7010,  441,  441, 5128,   41,   43,  448, 8747, 8747, 9224,                            │ │
│ │ │      9224, 6819, 6819, 2313,   41,   27,  474, 2928, 8060, 7443,   50,                            │ │
│ │ │   │    10,   27, 5248, 9005, 3675, 3675, 5932, 5932, 9768, 9768, 9945,                            │ │
│ │ │   │    26,   26,   26,  288,  262,  131,   10,   27,  282,  385,  385,                            │ │
│ │ │      1863, 2465, 2465,  230,  206,  230,  230,  230,  206,  206, 3317,                            │ │
│ │ │   │   206, 3317, 3317,  206,  206,  206,  206, 4653, 5196,  206,  206,                            │ │
│ │ │   │   206,  206, 5196,  206,  206,   65,  822,  822,  206, 1278, 1278,                            │ │
│ │ │   │   147,  147, 8395, 8395,  383, 5064,   10,    5,  651, 1169, 3504,                            │ │
│ │ │      3504,   41,   41,   40,   41,   10,   41,  399, 1460, 5694, 4808,                            │ │
│ │ │      4808, 9875, 9875, 9020, 9020, 4797,  575,  575,  685,  121,  292,                            │ │
│ │ │      2344, 3696,  686, 5640,  122,   41,   10,    5,  399, 6021, 8586,                            │ │
│ │ │      7934, 7934, 3305, 3305, 9765, 6564, 1239,  326, 2723, 7543, 8544,                            │ │
│ │ │      8399, 5508,   71,  100,    5, 6881, 5853, 5563, 5819,   41,  483,                            │ │
│ │ │      8671, 3948, 6699,   59,   59,   59,   41,  142,  830, 7429,  215,                            │ │
│ │ │   │   215,   77, 5067, 9090, 7484, 2120,   41,   17, 1793, 5303, 1335,                            │ │
│ │ │      7518,    3,   41,   10,   41, 7078, 5143, 5143, 8293, 8293, 8293,                            │ │
│ │ │      8293, 8919, 8919, 8919, 8919, 8919, 2286, 2286, 2286, 1093, 1093,                            │ │
│ │ │      1093,  894,  825,   97,   97,  230,  147,  206,  206,  230,  206,                            │ │
│ │ │   │   206, 1743, 6032, 3317,  206,  206,  206,  206,  206,  206,  206,                            │ │
│ │ │   │   206,  206,  206,  206,  206,  206,  206, 3252,  206,  206,  206,                            │ │
│ │ │      3252,  206,  206,  822, 1278,  822, 1278, 5772, 2305,  302, 5064,                            │ │
│ │ │      5064,  402, 1506, 7907, 6320, 2016,  579,   27,   27, 5727, 3265,                            │ │
│ │ │   │    41,   41,   41,   41, 5119, 6557, 6557, 3963, 3963, 3963, 3963,                            │ │
│ │ │      3767,   26,   26,   26,   26,   26,   26,   26,   26,  894,  894,                            │ │
│ │ │   │   138,  138,  206,  230,  230,  230,  230,  206,  193,  206,  193,                            │ │
│ │ 

In [56]:
text = """With me, brave the tumultuous seas, claim treasures untold, and send foes to their watery 
doom"""
kwargs["semantic_min_eos_p"] = 0.2 
kwargs["text_prompt"] = text
generation.set_seed(37)
kwargs['output_full']  = True
kwargs['output_dir'] = "seed37"
kwargs["history_prompt"] = None
#kwargs['history_prompt'] = 'custom_speakers/en_fiery.npz'
barkifull, audio_arr_segments_barki = api.generate_audio_barki(text, **kwargs)

set_seed Enabling deterministic algorithms                                                        ]8;id=679719;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=739196;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#929\929]8;;\

set_seed Set seed to 37                                                                           ]8;id=647827;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=697756;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#944\944]8;;\

generate_audio_barki {'text': 'With me, brave the tumultuous seas, claim treasures untold, and send foes ]8;id=540035;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=37163;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#362\362]8;;\
to their watery \ndoom', 'kwargs': {'text_prompt': 'With me, brave the tumultuous seas, claim treasures            
untold, and send foes to their watery \ndoom', 'list_speakers': None, 'dry_run': False,                            
'text_splits_only': False, 'history_prompt': None, 'prompt_file': None,                                            
'split_input_into_separate_prompts_by': None, 'split_input_into_separate_prompts_by_value': None,                  
'always_save_speaker': True, 'output_iterations': 1, 'output_filename': None, 'output_dir': 'seed37',              
'hoarder_mode': True, 'extra_stats': False, 'text_use_gpu': True, 'text_use_small': False,                         
'coarse_use_gpu': True, 'coarse_use_small': False, 'fine_use_gpu': True, 'fine_use_small': False,                  
'codec_use_gpu': True, 'force_reload': False, 'GLOBAL_ENABLE_MPS': None, 'USE_SMALL_MODELS': None,                 
'OFFLOAD_CPU': None, 'text_temp': 0.7, 'waveform_temp': 0.7, 'confused_travolta_mode': None, 'silent':             
False, 'seed': None, 'stable_mode_interval': 1, 'single_starting_seed': None,                                      
'split_character_goal_length': 145, 'split_character_max_length': 190, 'split_character_jitter': 0,                
'add_silence_between_segments': 0.0, 'split_each_text_prompt_by': None,                                            
'split_each_text_prompt_by_value': None, 'extra_confused_travolta_mode': None,                                     
'semantic_history_starting_weight': 1.0, 'semantic_history_future_weight': 1.0,                                    
'semantic_prev_segment_weight': 0.5, 'coarse_history_starting_weight': 1.0,                                        
'coarse_history_future_weight': 0.5, 'coarse_prev_segment_weight': 0.5, 'fine_history_starting_weight':            
1.0, 'fine_history_future_weight': 0.0, 'fine_prev_segment_weight': 0.0,                                           
'custom_audio_processing_function': None, 'use_smaller_models': False, 'semantic_temp': 0.7,                       
'semantic_top_k': None, 'semantic_top_p': None, 'semantic_min_eos_p': 0.2,                                         
'semantic_max_gen_duration_s': None, 'semantic_allow_early_stop': True, 'semantic_use_kv_caching': True,           
'semantic_seed': None, 'semantic_history_oversize_limit': None, 'coarse_temp': 0.7, 'coarse_top_k':                
None, 'coarse_top_p': None, 'coarse_max_coarse_history': 630, 'coarse_sliding_window_len': 60,                     
'coarse_kv_caching': True, 'coarse_seed': None, 'coarse_history_time_alignment_hack': -2, 'fine_temp':             
0.5, 'fine_seed': None, 'render_npz_samples': False, 'loglevel': 'WARNING', 'output_full': True}}                  

before load all defaults


generate_audio_barki {'text': 'With me, brave the tumultuous seas, claim treasures untold, and send foes ]8;id=827442;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=113095;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#366\366]8;;\
to their watery \ndoom', 'kwargs': {'text_prompt': 'With me, brave the tumultuous seas, claim treasures            
untold, and send foes to their watery \ndoom', 'list_speakers': None, 'dry_run': False,                            
'text_splits_only': False, 'history_prompt': None, 'prompt_file': None,                                            
'split_input_into_separate_prompts_by': None, 'split_input_into_separate_prompts_by_value': None,                  
'always_save_speaker': True, 'output_iterations': 1, 'output_filename': None, 'output_dir': 'seed37',              
'hoarder_mode': True, 'extra_stats': False, 'text_use_gpu': True, 'text_use_small': False,                         
'coarse_use_gpu': True, 'coarse_use_small': False, 'fine_use_gpu': True, 'fine_use_small': False,                  
'codec_use_gpu': True, 'force_reload': False, 'GLOBAL_ENABLE_MPS': None, 'USE_SMALL_MODELS': None,                 
'OFFLOAD_CPU': None, 'text_temp': 0.7, 'waveform_temp': 0.7, 'confused_travolta_mode': None, 'silent':             
False, 'seed': None, 'stable_mode_interval': 1, 'single_starting_seed': None,                                      
'split_character_goal_length': 145, 'split_character_max_length': 190, 'split_character_jitter': 0,                
'add_silence_between_segments': 0.0, 'split_each_text_prompt_by': None,                                            
'split_each_text_prompt_by_value': None, 'extra_confused_travolta_mode': None,                                     
'semantic_history_starting_weight': 1.0, 'semantic_history_future_weight': 1.0,                                    
'semantic_prev_segment_weight': 0.5, 'coarse_history_starting_weight': 1.0,                                        
'coarse_history_future_weight': 0.5, 'coarse_prev_segment_weight': 0.5, 'fine_history_starting_weight':            
1.0, 'fine_history_future_weight': 0.0, 'fine_prev_segment_weight': 0.0,                                           
'custom_audio_processing_function': None, 'use_smaller_models': False, 'semantic_temp': 0.7,                       
'semantic_top_k': None, 'semantic_top_p': None, 'semantic_min_eos_p': 0.2,                                         
'semantic_max_gen_duration_s': None, 'semantic_allow_early_stop': True, 'semantic_use_kv_caching': True,           
'semantic_seed': None, 'semantic_history_oversize_limit': None, 'coarse_temp': 0.7, 'coarse_top_k':                
None, 'coarse_top_p': None, 'coarse_max_coarse_history': 630, 'coarse_sliding_window_len': 60,                     
'coarse_kv_caching': True, 'coarse_seed': None, 'coarse_history_time_alignment_hack': -2, 'fine_temp':             
0.5, 'fine_seed': None, 'render_npz_samples': False, 'loglevel': 'WARNING', 'output_full': True}}                  

...

100%|██████████| 100/100 [00:05<00:00, 17.48it/s]


generate_coarse {'x_semantic': array([ 131,   10,   10,  266,  266,  266,  206,  206,  206,  206, ]8;id=151412;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=839049;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#460\460]8;;\
206,                                                                                                               
         65,  206,   65,  344,   65,  344,  147,  147,  147,  147, 1271,                                           
       1489, 7907, 5274, 8871,  579,  107,  107, 5727, 1620,   41,   41,                                           
         41, 5119, 6557, 6557, 3963, 3963, 3767, 3767,   26,   26,   26,                                           
         26,   26,   26,  894,  736,  894,   10,  230,  230,  230,   10,                                           
        206, 6032,  206, 6032,  147,  206,  206,  206,  206,   10,  206,                                           
        206, 3252, 2966,  147,  147, 1278,  147,  147,  147,  147,  147,                                           
        991,  991,  402,   10,   41,  245, 8969, 4860, 6323, 4099, 4868,                                           
       2244, 2244,  250, 9768, 1307, 4388, 9351, 7895,  178,  166, 1620,                                           
       1573, 6274, 7095, 7784,  210,   50,   10,    5, 2837, 3368, 3415,                                           
         41,   41,   41, 3135, 2065, 3240, 4579, 9792, 3400,  210,   10,                                           
        232,  401,  401, 9502, 2402, 2312, 2312, 3664, 1859, 5422, 1796,                                           
       3917,  880,   59,   59,   28,   28,   28,   28,   28,   28,   28,                                           
       1133, 3195, 6155, 6155, 3182, 8522, 8522, 3767, 3767,   44,   44,                                           
       4775,   44,   44, 9839, 8142, 3753, 5986, 1540,   59,   59,   28,                                           
        107,  589,  589,   10,   10,   27,  399,  287, 1460,  927, 9763,                                           
       4600, 6185, 6185, 7304, 3301, 4149, 5293, 5293, 5293,  911, 1544,                                           
         41,   41,  489,  850,  547,  210,   10,    5,  265,  401, 2739,                                           
       2773, 2848, 3840, 3890, 3890, 1450, 9403, 4146, 4146,  190,  190,                                           
       2280, 4394, 6022, 6022, 1517,  153,   71,   92,   92,  107, 1835,                                           
       3454, 3454, 3700, 1948, 1948,  444, 1315,  171,  171,  171,  489,                                           
        489,  489,  210,   10,    5,    5, 9367, 9367, 8498, 2745, 2745,                                           
       6696, 2779, 2779, 2779, 2779, 9454, 9454, 9454, 9454, 8417, 7006,                                           
       6672, 8270, 6018, 6018, 6139, 6139,  227, 6047,  441, 5647,   10,                                           
         10, 1096, 1096,   10,   10,  266,  206,  206,  206,  206,  206,                                           
        206,   10,  206,  206,  206,  206,   65,   65,  344,  147,  147,                                           
         57,   41,   57,  604, 5851, 8604, 5673, 4119,  171,   41,  171,                                           
        489,   59,   28,   28,   28,   28,   28,   10,   28, 6801, 9235,                                           
       3198, 1858,   29, 3793, 1315,  171,  171,  171,  171,  171,  489,                                           
        181,  181,   28,   28,   10,  337, 8963, 6412, 6412, 1001, 1001,                                           
       4816, 5301, 6303, 5301, 1667, 8525, 8525, 7365, 7365, 1647, 1647,                                           
         92,   59,   31,   31,   67,   10,    5, 8911, 5853, 5563, 5563,        

100%|██████████| 22/22 [00:15<00:00,  1.47it/s]


generate_fine {'x_coarse_gen': array([[876, 738, 855, ..., 408, 408, 738],                        ]8;id=433174;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=78803;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#626\626]8;;\
       [700, 841, 544, ..., 424, 424, 544]]), 'history_prompt': None, 'temp': 0.5, 'silent':                       
False}                                                                                                             

100%|██████████| 1/1 [00:04<00:00,  4.00s/it]


In [57]:
#inspect(barkifull)

for x in barkifull.keys():
    print(f"{x} {len(barkifull[x])}")

api.save_as_prompt("custom_speakers/seed37_barki.npz",barkifull)

semantic_prompt 429

coarse_prompt 2

fine_prompt 8

In [59]:
Audio(audio_arr_segments_barki, rate=generation.SAMPLE_RATE)

In [ ]:

def generate_audio_barki(
    text: str,
    **kwargs,
):
    """Generate audio array from input text.

    Args:
        text: text to be turned into audio
        history_prompt: history choice for audio cloning
        text_temp: generation temperature (1.0 more diverse, 0.0 more conservative)
        waveform_temp: generation temperature (1.0 more diverse, 0.0 more conservative)
        silent: disable progress bar
        output_full: return full generation to be used as a history prompt


    Returns:
        numpy audio array at sample frequency 24khz
    """
    logger.debug(locals())
    print("before load all defaults")
    kwargs = load_all_defaults(**kwargs)

    logger.debug(locals())
    print("after load all defaults")
    history_prompt = kwargs.get("history_prompt", None)
    text_temp = kwargs.get("text_temp", None)
    waveform_temp = kwargs.get("waveform_temp", None)
    silent = kwargs.get("silent", None)
    output_full = kwargs.get("output_full", None)

    global gradio_try_to_cancel
    global done_cancelling

    seed = kwargs.get("seed",None)
    if seed is not None:
        generation.set_seed(seed)

    ## TODO seperate stage seeds

    ## Semantic Options
    semantic_temp = text_temp
    if kwargs.get("semantic_temp", None):
        semantic_temp = kwargs.get("semantic_temp")

    semantic_seed = kwargs.get("semantic_seed",None)
    if semantic_seed is not None:
        generation.set_seed(semantic_seed)


    if gradio_try_to_cancel:
        done_cancelling = True
        return None, None
    semantic_tokens = call_with_non_none_params(
        generate_text_semantic,
        text=text,
        history_prompt=history_prompt,
        temp=semantic_temp,
        top_k=kwargs.get("semantic_top_k", None),
        top_p=kwargs.get("semantic_top_p", None),
        silent=silent,
        min_eos_p = kwargs.get("semantic_min_eos_p", None),
        max_gen_duration_s = kwargs.get("semantic_max_gen_duration_s", None),
        allow_early_stop = kwargs.get("semantic_allow_early_stop", True),
        use_kv_caching=kwargs.get("semantic_use_kv_caching", True),
    )
    
    if gradio_try_to_cancel:
        done_cancelling = True
        return None, None

    ## Coarse Options
    coarse_temp = waveform_temp
    if kwargs.get("coarse_temp", None):
        coarse_temp = kwargs.get("coarse_temp")

    coarse_seed = kwargs.get("coarse_seed",None)
    if coarse_seed is not None:
        generation.set_seed(coarse_seed)
        
    
    if gradio_try_to_cancel:
        done_cancelling = True
        return None, None
    
    coarse_tokens = call_with_non_none_params(
        generate_coarse,
        x_semantic=semantic_tokens,
        history_prompt=history_prompt,
        temp=coarse_temp,
        top_k=kwargs.get("coarse_top_k", None),
        top_p=kwargs.get("coarse_top_p", None),
        silent=silent,
        max_coarse_history=kwargs.get("coarse_max_coarse_history", None),
        sliding_window_len=kwargs.get("coarse_sliding_window_len", None),
        use_kv_caching=kwargs.get("coarse_kv_caching", True),
    )

    fine_temp = kwargs.get("fine_temp", 0.5)

    fine_seed = kwargs.get("fine_seed",None)
    if fine_seed is not None:
        generation.set_seed(fine_seed)

    if gradio_try_to_cancel:
        done_cancelling = True
        return None, None
    fine_tokens = call_with_non_none_params(
        generate_fine,
        x_coarse_gen=coarse_tokens,
        history_prompt=history_prompt,
        temp=fine_temp,
        silent=silent,
    )

    # do we ever care about setting this seed? Probably not? You can always just decode it again

    if gradio_try_to_cancel:
        done_cancelling = True
        return None, None
    audio_arr = codec_decode(fine_tokens)
    full_generation = {
        "semantic_prompt": semantic_tokens,
        "coarse_prompt": coarse_tokens,
        "fine_prompt": fine_tokens,
    }

    if gradio_try_to_cancel:
        done_cancelling = True
        return None, None
    
    hoarder_mode = kwargs.get("hoarder_mode", None)
    total_segments = kwargs.get("total_segments", 1)
    if hoarder_mode and (total_segments > 1):
        kwargs["text"] = text
        write_one_segment(audio_arr, full_generation, **kwargs)

    if output_full:
        return full_generation, audio_arr
    
    return audio_arr

In [60]:

generation.set_seed(37)
#kwargs['history_prompt'] = 'custom_speakers/en_fiery.npz'



#fullgen, gen_audio_orig_segs  = api.generate_audio(text, output_full=True, **kwargs)


fullgen, gen_audio_orig_segs  = api.generate_audio(text, output_full=True)

set_seed Enabling deterministic algorithms                                                        ]8;id=679719;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=739196;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#929\929]8;;\

set_seed Set seed to 37                                                                           ]8;id=647827;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=697756;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#944\944]8;;\

generate_text_semantic {'text': 'With me, brave the tumultuous seas, claim treasures untold, and  ]8;id=540035;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=37163;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#302\302]8;;\
send foes to their watery \ndoom', 'history_prompt': None, 'temp': 0.7, 'top_k': None, 'top_p':                    
None, 'silent': False, 'min_eos_p': 0.2, 'max_gen_duration_s': None, 'allow_early_stop': True,                     
'use_kv_caching': True}                                                                                            

100%|██████████| 100/100 [00:04<00:00, 20.12it/s]


generate_coarse {'x_semantic': array([ 131,   10,   10,  266,  266,  266,  206,  206,  206,  206, ]8;id=827442;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=113095;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#460\460]8;;\
206,                                                                                                               
         65,  206,   65,  344,   65,  344,  147,  147,  147,  147, 1271,                                           
       1489, 7907, 5274, 8871,  579,  107,  107, 5727, 1620,   41,   41,                                           
         41, 5119, 6557, 6557, 3963, 3963, 3767, 3767,   26,   26,   26,                                           
         26,   26,   26,  894,  736,  894,   10,  230,  230,  230,   10,                                           
        206, 6032,  206, 6032,  147,  206,  206,  206,  206,   10,  206,                                           
        206, 3252, 2966,  147,  147, 1278,  147,  147,  147,  147,  147,                                           
        991,  991,  402,   10,   41,  245, 8969, 4860, 6323, 4099, 4868,                                           
       2244, 2244,  250, 9768, 1307, 4388, 9351, 7895,  178,  166, 1620,                                           
       1573, 6274, 7095, 7784,  210,   50,   10,    5, 2837, 3368, 3415,                                           
         41,   41,   41, 3135, 2065, 3240, 4579, 9792, 3400,  210,   10,                                           
        232,  401,  349, 9502, 2312, 2312, 4408, 4408, 1796,  880,  207,                                           
         59,   28,   28,   28,   28,   10,   28, 1696, 3195, 6155, 3182,                                           
       8522, 6089,  215,  215, 4116, 8541, 8541, 3767, 4139,   44, 9839,                                           
       9839, 9339, 5986, 5986,  137,  107,   28,   28,  107,  667, 2672,                                           
         10,   10,  266, 1147,  206,  107,  206,  206,  206,  206,  147,                                           
       2127, 2156,  147, 5008,   27,   27,  399,  399, 3188, 6289, 3969,                                           
       6185, 6185, 6185, 4475, 4149, 2425, 2425, 4145,  717,  633,   41,                                           
        171,  489,  210,  100,    5,  401, 2361, 2739, 2773, 2848, 3840,                                           
       3890, 1450, 1450,   92,   59,  190, 9578, 2280, 6022, 6022,  153,                                           
       5191,   92,   59,   28,  148, 3454, 3700, 3700, 1948,  444, 5693,                                           
        171,  171,  489,  489,  210,   10,    5,  265, 9367, 9367, 2745,                                           
       2745, 6696, 2745, 6696, 6696, 2779, 2779, 9454, 9454, 8417, 8417,                                           
       5500, 8270, 7183,   10, 6139,  227, 1980, 6047,  441, 5647,   10,                                           
         10, 1096, 1096,   10,  206,  206,  206,  206,  206,  206,  206,                                           
        206,   65,  206,  344,   65,  344,  147,  147, 3888,   57,  567,                                           
       1973,  898,  171,  122,   59,   28,   28,   10,   28, 1133, 9235,                                           
       3198, 1858,   29, 3793, 1315,  171,  171,  171,  171,  181,   59,                                           
         28,   10,   41, 1966,  323, 6412, 1001, 1001, 4816, 5301, 6303,                                           
       6303, 1667, 1667, 1667, 7365, 7365, 1647, 1647,   92,   59,   59,                                           
         31,   31,   10,    5, 5853, 5563, 5563,  521,   41,  483, 9968,                                           
       5839, 5839, 3177, 1493,  393,  429,   41,   41, 1904, 3277,  774,        

100%|██████████| 20/20 [00:13<00:00,  1.47it/s]


generate_fine {'x_coarse_gen': array([[1019, 1019, 1019, ...,  432,  432,  724],                  ]8;id=295619;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=970216;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#626\626]8;;\
       [ 829,  841,  928, ...,  894,  894,  765]]), 'history_prompt': None, 'temp': 0.5,                           
'silent': True}                                                                                                    

In [52]:

Audio(gen_audio_orig_segs, rate=generation.SAMPLE_RATE)



In [61]:
for k in fullgen:
    print(f"k: {k}")
    print(f"len: {len(fullgen[k])}")

k: semantic_prompt

len: 390

k: coarse_prompt

len: 2

k: fine_prompt

len: 8

In [31]:
api.render_npz_samples("custom_speakers", start_from="semantic")

Rendering samples for speakers in: custom_speakers
  Rendering audio for custom_speakers/hark_woman.npz to custom_speakers/hark_woman_1.wav


write_seg_wav   .wav saved to custom_speakers/hark_woman_1.wav                                           ]8;id=552246;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=649487;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#325\325]8;;\

  Rendering audio for custom_speakers/hark1.npz to custom_speakers/hark1_1.wav


write_seg_wav   .wav saved to custom_speakers/hark1_1.wav                                                ]8;id=9210;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=7946;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#325\325]8;;\

  Rendering audio for custom_speakers/en_fiery.npz to custom_speakers/en_fiery_1.wav


write_seg_wav   .wav saved to custom_speakers/en_fiery_1.wav                                             ]8;id=71943;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=741508;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#325\325]8;;\

In [38]:
Audio(gen_audio_orig_segs, rate=generation.SAMPLE_RATE)

In [ ]:


def generate_audio(
    text: str,
    history_prompt: Optional[Union[Dict, str]] = None,
    text_temp: float = 0.7,
    waveform_temp: float = 0.7,
    silent: bool = False,
    output_full: bool = False,
):
    """Generate audio array from input text.

    Args:
        text: text to be turned into audio
        history_prompt: history choice for audio cloning
        text_temp: generation temperature (1.0 more diverse, 0.0 more conservative)
        waveform_temp: generation temperature (1.0 more diverse, 0.0 more conservative)
        silent: disable progress bar
        output_full: return full generation to be used as a history prompt

    Returns:
        numpy audio array at sample frequency 24khz
    """
    semantic_tokens = text_to_semantic(
        text,
        history_prompt=history_prompt,
        temp=text_temp,
        silent=silent,
    )
    out = semantic_to_waveform(
        semantic_tokens,
        history_prompt=history_prompt,
        temp=waveform_temp,
        silent=silent,
        output_full=output_full,
    )
    if output_full:
        full_generation, audio_arr = out
        return full_generation, audio_arr
    else:
        audio_arr = out
    return audio_arr

## ADDED BELOW

In [17]:
Audio(audio_arr_segments_barki, rate=generation.SAMPLE_RATE)

In [49]:
kwargs['history_prompt'] = 'custom_speakers/hark_woman.npz'

In [ ]:
print(kwargs)

In [45]:
api.render_npz_samples("custom_speakers", start_from="semantic_prompt")

Rendering samples for speakers in: custom_speakers


generate_coarse {'x_semantic': array([ 206,  206,  206,  206,  206,  206,  206,  206,  206,  206, ]8;id=151412;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=839049;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#460\460]8;;\
206,                                                                                                               
        206,  206,  206,  206,   91,  206,  206,  206,  206,  206,  206,                                           
        206,  206,  206, 1334,  206,  206,  206,  206,  206, 1334, 3252,                                           
         65, 1334, 1334, 1905, 1905, 1905,  258,  134, 1905,  134,  258,                                           
        134,  134,  134,  134,  134,  134,  258,  134,  134,  134,  134,                                           
        134,  134,  134,  134,  134,  134,  134,  134,  134, 3350, 3350,                                           
        134,  881, 3350, 6206,  147, 6961,  302,  608, 9877, 5214, 3427,                                           
       3427, 1947, 3427,  133, 8308,  133, 2665, 7532, 7532, 7532, 4155,                                           
       4155, 2323, 4155,  990,   26, 1828,   26,  894,  175,  175,  736,                                           
       7308,   97,   97, 2640, 2640, 2640,   56,  230, 1083, 2009,  147,                                           
       1083, 9260,  147, 9344, 9344, 3584, 9344, 5636,  402,  402, 2574,                                           
       7692, 8119, 2571,   59,   59,  181,   28,  923,  923,  923, 1284,                                           
         28,   28,  337, 3620, 1225, 1680, 6523, 5714, 2896, 3787,  171,                                           
        171,  255,   41,    5,  282, 9717, 1494, 2036, 2267,   41,   41,                                           
         41,   41, 1436,  402,   41,   41, 9532, 6903, 6903, 6903,   92,                                           
         28,   28, 1133, 3190, 8249, 8249,  874,   50,   27,   99,  401,                                           
       8458, 1291, 3745, 5218,  117,   41,  402, 7990, 1025,   10,  604,                                           
       5246, 4566, 6119, 6119, 4187,  178,   17,  452,  626,  429,   41,                                           
        429, 1978, 8024, 8024, 8631,  178,  210,  210,  211,  441,   10,                                           
          5,    5,  282, 3538, 4737, 6664, 2135, 3755,  233,  233, 2095,                                           
        232,  100,   27,  532,  401,  142,  107,  142,  142,  142,  830,                                           
        577,  577, 5211, 2465, 1233, 2784,  763, 2784, 1083,  230, 1083,                                           
       1083,  528,  206, 9260,  528, 1613, 9260,  528, 6961, 3584, 3584,                                           
       6378, 9893,  429,   41,  429, 7168, 9742, 8512, 8512, 5651, 8308,                                           
       2685, 8308, 2665, 2665, 7038, 4155, 2323,  163,  163, 5012, 5012,                                           
         92,   59,   59,  136, 4063, 2595, 2595, 7352, 9725, 6445, 6445,                                           
         41,   41,   41,  130,   41,  483, 5373, 4640, 2655, 8891, 1040,                                           
       6358,  808,  808,  117,   50,   41,   10,   41, 5991,  203, 1831,                                           
        621,  621,  981,  529,   59,   59,   59,   28,   28,   28,   28,                                           
         28,   28,   28,  107,  107,  223,  385,  138,   10,  206,  266,                                           
        459, 3316, 1444,  206,  206,  206,  206,  206,  206,  206,  206,                                           
        206,  206,  206,  206,  206,  206,  206, 3252,   91,  206,  206,        

100%|██████████| 29/29 [00:25<00:00,  1.15it/s]


generate_fine {'x_coarse_gen': array([[475, 475, 475, ..., 572, 566, 910],                        ]8;id=433174;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=78803;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#626\626]8;;\
       [913, 913, 580, ..., 730, 271, 493]]), 'history_prompt': None, 'temp': 0.5, 'silent':                       
True}                                                                                                              

  Rendering audio for custom_speakers/hark1.npz to custom_speakers/hark1_1.wav


...

generate_coarse {'x_semantic': array([ 147, 6242,  302, 1106,  401, 2739, 9511, 9924, 9924, 9924, ]8;id=71943;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=741508;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#460\460]8;;\
2092,                                                                                                              
        122,   59,  417,  107,  884, 3446, 5027, 2908, 2908, 8010, 7410,                                           
       5201,   92,   59,  148,  849,  739,   10, 2436, 1134, 6025, 6025,                                           
       1080, 5584,  370,  370, 4663,  756,  579,  579, 9648, 6713, 1252,                                           
       1850,  171,  210,  181,   10,   27,  399, 3805, 9883, 9692, 8600,                                           
        211,   43,   27, 3033, 6949,  326,  326, 9690, 3700, 7137,  995,                                           
       1315,  321, 6676, 1263,  880,  207,   59,   59,   59, 5356,   67,                                           
        196,  718, 8607, 2581, 1990,  518,  452, 1119, 1058, 7769, 6507,                                           
       6507,  298,   17, 2186, 2186,  518, 7480,  648,   41, 4967,  774,                                           
        774, 6247, 4417,  620,   92,   59, 1765, 3454, 7891, 7891, 6025,                                           
       2818, 1482,  326, 2258, 3654, 6627, 5986,   92,   92,  315, 2466,                                           
       1079,  441,  402,   10,   10,  282, 8136, 9329, 9329, 1803, 1859,                                           
       9009, 9009, 5646, 2313,   41,  474, 2928, 2568, 6259,   12,   12,                                           
         59,   59,   28,  142, 4839, 7465, 7465, 9790, 4718,  250,   26,                                           
        122,   59,  181,   28,   41, 9866, 4051, 3166, 3281, 3281,   85,                                           
         85,   85,   85,  175,  736,  230,  230,   10,   56,  230, 5008,                                           
         56,  206,  206,  206,  206,  206,  206,  206,  206,  206,  206,                                           
        206,  206,  206,  206, 3252,  206,   91, 3252,  206,  206, 3252,                                           
        206,   65,   65,   65,   65,   65,  413,  147, 3208,   57, 1134,                                           
       6025, 6025,  648,  326, 4770, 4417, 4417,  620,   59,  148, 7035,                                           
       6957, 1541, 1461, 6985, 4705, 7763, 7233,   64,  298, 2783, 7387,                                           
        210,   50,   10,   27, 1169, 3504, 1821,  321,  321, 1227,  402,                                           
        321,   10,   41,  652, 3158,  847, 8396, 8396, 5315, 1044,  541,                                           
        100,   27,  617, 7552,   10,  282,  790,  395, 7686,  138,   10,                                           
        266, 1752,  767,  767, 1075, 3490,   36, 5729, 9353, 9353, 3223,                                           
        508, 8785, 5358, 2419, 8581, 1275,  122,   59,   28,   28,   28,                                           
         28, 1966,  323, 5506, 1568,  468,  118,   50,   27,   27, 8826,                                           
       8826, 1315,  171,  171, 1732,   41,   41, 8265,  730, 4623, 4623,                                           
       2873, 2873, 9145, 9145,  389, 7194, 9742, 1653,  187, 3287, 2561,                                           
        376,  376, 1339, 1425,  589,   27, 1041, 6107, 1026, 7602,  171,                                           
        648,  429,   41, 9181, 8366, 4690, 4690,  250, 9351,  288,  245,                                           
       9231, 9182, 3891, 2339,   92,   92,  107, 3101, 3454, 8859, 6202,         

100%|██████████| 35/35 [00:31<00:00,  1.09it/s]


generate_fine {'x_coarse_gen': array([[408, 408, 408, ..., 408, 408,  62],                        ]8;id=368965;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py\generation.py]8;;\:]8;id=553125;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/generation.py#626\626]8;;\
       [424, 424, 424, ..., 518, 518, 424]]), 'history_prompt': None, 'temp': 0.5, 'silent':                       
True}                                                                                                              

  Rendering audio for custom_speakers/en_fiery.npz to custom_speakers/en_fiery_1.wav


write_seg_wav   .wav saved to custom_speakers/en_fiery_1.wav                                             ]8;id=763391;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py\api.py]8;;\:]8;id=14160;file:///home/jon/mamba_projects/bark_postfixes/bark/bark_infinity/api.py#325\325]8;;\

In [ ]:
Audio(audio_arr_segments_barki, rate=generation.SAMPLE_RATE) 

In [ ]:
generation.preload_models()

In [ ]:
text = """
Hark! I, the phantom visage of Edward Teach, rise from the 
abyss, forever bound to the briny depths. With me, brave the tumultuous seas, claim treasures untold,
and send foes to their watery doom.
"""

In [ ]:
history_prompt = np.load("pirates/base/pirate.npz")
from rich import inspect

In [ ]:
for key in history_prompt.keys():
    length = len(history_prompt[key])
    print(f"key: {key}, length: {length}")
    inspect(history_prompt[key], title=f"{key} ({length})")


In [ ]:
new_semantic = np.hstack([x_semantic_history, x_semantic]).astype(np.int32)

semantic_prompt = history_prompt["semantic_prompt"]
midpoint = len(semantic_prompt) // 2
new_semantic_first_half = semantic_prompt[:midpoint].astype(np.int32)


Instead I would like new_semantic to be half the size of x_semantic_history, just the last half of the space.

In [ ]:
    if length > 0:
        for sub_key in history_prompt[key].keys():
            print(f"  {sub_key}={history_prompt[key][sub_key]}")


In [ ]:

gen_minor_variants = 20
import random

npz_file = "pirate.npz"
npz_directory = "pirates/base"
npz_filepath = "pirates/base/pirate.npz"

semantic_prompt = history_prompt["semantic_prompt"]
original_semantic_prompt = semantic_prompt.copy()
starting_point = 128
ending_point = len(semantic_prompt) - starting_point



points = np.linspace(starting_point, ending_point, gen_minor_variants)
      
for starting_point in points:
    starting_point = int(starting_point)
    print(starting_point)

    new_semantic_from_beginning = original_semantic_prompt[:starting_point].astype(np.int32)
    new_semantic_from_ending = original_semantic_prompt[starting_point:].astype(np.int32)

    for semantic_prompt in [new_semantic_from_beginning, new_semantic_from_ending]:
        
        print(f"len(semantic_prompt): {len(semantic_prompt)}")
        print(f"starting_point: {starting_point}, ending_poinst: {ending_point}")  

        temp_coarse = random.uniform(0.5, 0.9)
        top_k_coarse = None if random.random() < 1/3 else random.randint(50, 100)
        top_p_coarse = None if random.random() < 1/3 else random.uniform(0.8, 0.95)

        max_coarse_history_options = [630, random.randint(500, 630), random.randint(60, 500)]
        max_coarse_history = random.choice(max_coarse_history_options)

        coarse_tokens = generation.generate_coarse(semantic_prompt, temp=temp_coarse, top_k=top_k_coarse, top_p=top_p_coarse, max_coarse_history=max_coarse_history)

        temp_fine = random.uniform(0.3, 0.7)
        fine_tokens = generation.generate_fine(coarse_tokens, temp=temp_fine)

        history_prompt_render_variant = {"semantic_prompt": semantic_prompt, "coarse_prompt": coarse_tokens, "fine_prompt": fine_tokens}

        try:
            audio_arr = generation.codec_decode(fine_tokens)
            base_output_filename = os.path.splitext(npz_file)[0] + f"_var_{i}.wav"
            output_filepath = os.path.join(npz_directory, base_output_filename)
            output_filepath = api.generate_unique_filepath(output_filepath)
            print(f"  Rendering minor variant voice audio for {npz_filepath} to {output_filepath}")
            api.write_seg_wav(output_filepath, audio_arr)

            api.write_seg_npz(output_filepath, history_prompt_render_variant)
        except:
            print(f"  <Error rendering audio for {npz_filepath}>")

### Set Text and Other Generation Options

In [ ]:
text = """
Hey, have you heard about this new text-to-audio model called "Bark"? 
It's like rain on your wedding day. It's a free ride when you've already paid. It's the good advice that you just didn't take.
And who would've thought? It figures.

Well, life has a funny way of sneaking up on you. When you think everything's okay and everything's going right. 
And life has a funny way of helping you out. When you think everything's gone wrong. 
And everything blows up in your face.

It's a traffic jam when you're already late. A "No smoking" sign on your cigarette break.
It's like ten thousand spoons when all you need is a knife. It's meeting the man of my dreams.
And then meeting his beautiful wife.

And isn't it ironic? Don't you think? A little too ironic.
And yeah, I really do think.
"""

# FOr split set split_character_goal_length and split_character_max_length
kwargs = {}

kwargs = config.load_all_defaults()
kwargs['text_prompt'] = text
kwargs['hoarder_mode'] = True
kwargs["output_dir"] = 'bark_samples'
kwargs["history_prompt"] = None
# kwargs["single_starting_seed"] = None # 
# If you set seed you might want manually call generation.set_seed(-1) after to disable deterministic generation settings 
# I'm not cleaning up after this paramater at the moment and I'm not sure on other side effects
kwargs["stable_mode_interval"] = 1 # 0 for continous, 2,3,4 for mixed
kwargs["split_character_goal_length"] = 90
kwargs["split_character_max_length"] = 130
# kwargs["output_iterations"] = 1
kwargs["add_silence_between_segments"] = .025 # See: https://github.com/suno-ai/bark/blob/main/notebooks/long_form_generation.ipynb but not great for songs or stable_mode_interval 0
kwargs["semantic_min_eos_p"] = 0.05 # 0.20 is default, lower means more likely to stotp


# not sure on overall effect so far from these, but for example:
kwargs["semantic_top_k"] = 50
kwargs["semantic_top_p"] = 0.95

### First Attempt

#### Before we run, let's double check out settings

In [ ]:
kwargs["dry_run"] = True # Check how the text is being split, don't actually run the model. 
full_generation_segments, audio_arr_segments, final_filename_will_be = api.generate_audio_long(**kwargs)

In [ ]:
# that's the output we expect to see, we didn't generate audio yet
# these text segments look a little small small so let's try this instead
kwargs["split_character_goal_length"] = 110
kwargs["split_character_max_length"] = 175

full_generation_segments, audio_arr_segments, final_filename_will_be = api.generate_audio_long(**kwargs)

#### Run Bark

In [ ]:
# These segement sizes look better so now so set dry_run to False to run for real
# Because we set hoarder_mode we can see the wav files for each segment in the Colab File Manager

kwargs["dry_run"] = False
full_generation_segments, audio_arr_segments, final_filename_will_be = api.generate_audio_long(**kwargs)

In [ ]:
print(f"  final wav at {final_filename_will_be}  ")
# (we see many wav because we set hoarder_mode, but one file will be the final product
# set hoarder_mode=False if you just want the file wav and aren't in explore mode

# or play here 
Audio(np.concatenate(audio_arr_segments), rate=generation.SAMPLE_RATE) 


In [ ]:
# because we set hoarder mode we also saved each segement as its own seperate sample with wav

!find "bark_samples/" -name "*.npz"

display_wav_files("bark_samples/")

### Second Attempt. Can we do better?

In [ ]:
# we used stable_mode_interval = 1, so the history_prompt does not evolve between segments
# even still the voices that are saved for each segment are one-generation different than the original history prompt
# this means they are a *little* bit different, and we may prefer one of them over the original
# for example maybe segment 2 was a little more clear, or had a particular emotion, we could use that segment's version as the speaker
# in the particular run I'm doing now, that segment ended with a little bit an interesting accent. I'm curious if I can bring that out more.

# (should probably rename the file to something sensible though)

kwargs["history_prompt"] = "/content/bark/bark_samples/Hey_have_you_heard_a-SPK-random.wav/002_Its_the_good_advice_-SPK-random.wav.npz"

In [ ]:
kwargs["text_prompt"] = f"I'm speaker number two. I'm the best speaker. Also I'm a free spirit. Let me evolve my voice with every step. Here's my version."
kwargs["text_prompt"] += text
kwargs["stable_mode_interval"] = 0 
kwargs["output_dir"] = "speaker_2_test"
kwargs["add_silence_between_segments"] = 0.0 # No silence, fully merge clips

kwargs["semantic_min_eos_p"] = 0.20 # Back to default, let Bark umm and ahh a bit
full_generation_segments, audio_arr_segments, final_filename_will_be = api.generate_audio_long(**kwargs)



In [ ]:
print(f"  final wave at {final_filename_will_be}")
Audio(np.concatenate(audio_arr_segments), rate=generation.SAMPLE_RATE) 

In [ ]:
# this clip probably got really weird after a full segments, fully feedbacking into itself. So kwargs["stable_mode_interval"] = 3 might be a good compromise

display_wav_files("speaker_2_test")

### Finding Our Voice

In [ ]:
# That final clip is an improvement, the random voice we got isn't bad but it's not quite doing our beautiful prose justice
# we could use an existing history_prompt, but let's try to summon a perfect speaker from the model
# we do that by generating many speakers randomly
# we could use our first segment text, in my experience there is a better method
# try to image: what type of text would be the context in voice I want to hear is likely to appear?
# then let's generate 20 sample clips from that

# TODO

In [ ]:

text="""
How many fucken morons can wave his arms and keep people in tempo? 
I was there to push people beyond what's expected of them. 
I believe that is an absolute necessity. 
Otherwise we're depriving the world of the next Louis Armstrong, 
or the next Charlie Parker. 
Have I told you that story about how Charlie Parker became Charlie Parker?
Parker's a young kid, pretty good on the Sax, 
gets up to play at a cutting session, 
and well, he fucks it up. 
And Jones nearly decapitates him for it, throws a cymbal at his head. 
And Charlie's laughed off stage. Cries himself to sleep that night. 
But the next morning, what does he do? He practices. 
And he practices and he practices and he practices. 
With one goal in mind - never to be laughed at again. 
And a year later he goes back to the Reno, 
and he steps up on that stage 
and he plays the best motherfucken solo the world has ever heard. 
So imagine if Jones had just said, 
"Well that's okay, Charlie, that was alright. Good job.” 
And Charlie thinks to himself, “Well, shit I did do a pretty good job.” 
End of story. That to me is an absolute tragedy. 
But that's just what the world wants now. 
No wonder Jazz is dying.

[He takes a sip of his drink.]
I tell you man, every Starbucks “Jazz” album, just proves my point really 
- there are no two words more harmful in the English language than “Good job”.

The truth is Andrew I… never really had a Charlie Parker. 
But I tried. I actually fucking tried. 
And that's more than most people ever do, 
and I will never apologise for how I tried.
"""

In [ ]:
import re
def apply_rule_to_prompt(regex, replacement, flags, text):

  
    re_flags = 0
    if 'MULTILINE' in flags:
        re_flags |= re.MULTILINE
    return re.sub(regex, replacement, text, flags=re_flags)

In [ ]:
regex = "[\.?,]"
replacement = " "
flags = "MULTILINE"

print(apply_rule_to_prompt(regex, replacement, flags, text))

In [ ]:
from typing import List
import re
import random 
from typing import Dict, Optional, Union
import logging
logger = logging.getLogger(__name__)
def split_text(text: str, split_type: Optional[str] = None, split_type_quantity = 1, split_type_string: Optional[str] = None, split_type_value_type: Optional[str] = None) -> List[str]:
    if text == '':
        return [text]

    # the old syntax still works if you don't use this parameter, ie
    # split_type line, split_type_value 4, splits into groups of 4 lines
    if split_type_value_type == '':
        split_type_value_type = split_type

    """
    if split_type == 'phrase':
        # print(f"Loading spacy to split by phrase.")
        nlp = spacy.load('en_core_web_sm')

        chunks = split_by_phrase(text, nlp)
        # print(chunks)
        return chunks
    """
    if split_type == 'string' or split_type == 'regex':

        if split_type_string is None:
            logger.warning(
                f"Splitting by {split_type} requires a string to split by. Returning original text.")
            return [text]

    split_type_to_function = {
        'word': split_by_words,
        'line': split_by_lines,
        # 'sentence': split_by_sentences,
        'string': split_by_string,
        #'random': split_by_random,
        # 'rhyme': split_by_rhymes,
        # 'pos': split_by_part_of_speech,
        'regex': split_by_regex,
    }

    if split_type in split_type_to_function:
        # split into groups of 1 by the desired type
        # this is so terrible even I'm embarassed, destroy all this code later, but I guess it does something useful atm
        segmented_text = split_type_to_function[split_type](text, split_type = split_type, split_type_quantity=1, split_type_string=split_type_string, split_type_value_type=split_type_value_type)
        final_segmented_text = []
        current_segment = ''
        split_type_quantity_found = 0
        for seg in segmented_text: # for each line, for example, we can now split by 'words' or whatever, as a counter for when to break the group
            current_segment += seg

            #print(split_type_to_function[split_type](current_segment, split_type=split_type_value_type, split_type_quantity=1, split_type_string=split_type_string))
            split_type_quantity_found = len(split_type_to_function[split_type](current_segment, split_type=split_type_value_type, split_type_quantity=1, split_type_string=split_type_string))
            print(f"I see {split_type_quantity_found} {split_type_value_type} in {current_segment}")
            if split_type_quantity_found >= split_type_quantity:
                final_segmented_text.append(current_segment)
                split_type_quantity_found = 0
                current_segment = ''
            
        return final_segmented_text

    logger.warning(
        f"Splitting by {split_type} not a supported option. Returning original text.")
    return [text]

def split_by_string(text: str, split_type: Optional[str] = None, split_type_quantity: Optional[int] = 1, split_type_string: Optional[str] = None, split_type_value_type: Optional[str] = None) -> List[str]:
    if split_type_string is not None:
        split_pattern = f"({split_type_string})"
        split_list = re.split(split_pattern, text)
        result = [split_list[0]]
        for i in range(1, len(split_list), 2):
            result.append(split_list[i] + split_list[i+1])
        return result
    else:
        return text.split()

def split_by_regex(text: str, pattern: str) -> List[str]:
    chunks = []
    start = 0

    for match in re.finditer(pattern, text):
        end = match.start()
        chunks.append(text[start:end].strip())
        start = end

    chunks.append(text[start:].strip())
    return chunks


def split_by_words(text: str, split_type: Optional[str] = None, split_type_quantity = 1, split_type_string: Optional[str] = None, split_type_value_type: Optional[str] = None) -> List[str]:
    words = text.split()
    return [' '.join(word for word in words)]
    #return [' '.join(words[i:i + split_type_quantity]) for i in range(0, len(words), split_type_quantity)]


def split_by_lines(text: str, split_type: Optional[str] = None, split_type_quantity = 1, split_type_string: Optional[str] = None, split_type_value_type: Optional[str] = None) -> List[str]:
    lines = [line + '\n' for line in text.split('\n') if line.strip()]
    return lines
    #return ['\n'.join(lines[i:i + split_type_quantity]) for i in range(0, len(lines), split_type_quantity)]

"""
def split_by_sentences(text: str, n: int, language="en") -> List[str]:
    seg = pysbd.Segmenter(language=language, clean=False)
    sentences = seg.segment(text)
    return [' '.join(sentences[i:i + n]) for i in range(0, len(sentences), n)]
"""

def load_text(file_path: str) -> Union[str, None]:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        logger.info(f"Successfully loaded the file: {file_path}")
        return content
    except FileNotFoundError:
        logger.error(f"File not found: {file_path}")
    except PermissionError:
        logger.error(f"Permission denied to read the file: {file_path}")
    except Exception as e:
        logger.error(
            f"An unexpected error occurred while reading the file: {file_path}. Error: {e}")
    return None


# Good for just exploring random voices, rethink api
"""
def split_by_random(text: str, split_type: Optional[str] = None, split_type_quantity = 1, split_type_string: Optional[str] = None, split_type_value_type: Optional[str] = None) -> List[str]:
    segments = split_text(text, split_type_string)
    chunks = []
    min_len = max(1, split_type_quantity - 2)
    max_len = split_type_quantity + 2
    while words:
        chunk_len = random.randint(min_len, max_len)
        chunk = ' '.join(words[:chunk_len])
        chunks.append(chunk)
        words = words[chunk_len:]
    return chunks
"""

In [ ]:
print(split_by_lines(text))

In [ ]:
split_text(text="A dig whe sdf lskfj.", split_type="word")

In [ ]:
print(split_text(text, split_type="line", split_type_quantity=4, split_type_value_type="line"))


In [ ]:
text = """
It's like rain on your wedding day.
It's a free ride when you've already paid.
It's the good advice that you just didn't take.
And who would've thought? It figures.
It's like rain on your wedding day.
It's a free ride when you've already paid.
It's the good advice that you just didn't take.
And who would've thought? It figures.
"""

result = split_text(text, split_type="line", split_type_quantity=3)
print(result)

result = split_text(text, split_type="word", split_type_quantity=4)
print(result)

In [ ]:
result = split_text(text, split_type="string", split_type_quantity=23, split_type_string = "the")
print(result)

In [ ]:
print(split_by_string("The dog went to the dog store and ate", split_type="string", split_type_quantity=23, split_type_string="dog"))


In [ ]:
split_text(text, split_type="line", split_type_quantity=3, split_type_value_type="word")